# <center>Big Data &ndash; Exercise 1</center>
## <center>Fall 2022 &ndash; Week 1 &ndash; ETH Zurich</center>

### Aims
- **After this exercise:** Understand the SQL language and its common query patterns.
- **Later in the semester:** Relate these language features and query patterns relative to other data shapes, technologies, and the languages designed to query them.



- **After this exercise:** Understand the 'table' data shape, normalization, and when they can (and should) be used.
- **Later in the semester:** Understand when you can (and should) throw all of this away!

### Prerequisites
In this exercise, you will brush-up the fundamental concepts of relational databases and SQL. If you haven't taken an introductory databases course (or want to refresh your knowledge) we recommend you to read the following:

Garcia-Molina, Ullman, Widom: Database Systems: The Complete Book. Pearson, 2. Edition, 2008. (Chapters 1, 2, 3, and 6) [Available in the ETH Library] [[Online]](https://ebookcentral.proquest.com/lib/ethz/detail.action?pq-origsite=primo&docID=5832965) [[Selected solutions]](http://infolab.stanford.edu/~ullman/dscbsols/sols.html)

### Database Set-up
Unlike last week's exercise, the dataset for this exercise might take a little bit longer to download and initialize. Please wait for the message `PostgreSQL init process complete; ready for start up` in the docker logs before proceeding!

As before, we set up our connection to the database and enable use of `%sql` and `%%sql`.

In [1]:
server='postgres'
user='postgres'
password='BigData1'
database='discogs'
connection_string=f'postgresql://{user}:{password}@{server}:5432/{database}'

In [2]:
%reload_ext sql
%sql $connection_string

ModuleNotFoundError: No module named 'sql'

In [5]:
%%sql
SELECT version();

UsageError: Cell magic `%%sql` not found.


## Exercise 1: Explore the dataset
We want to first understand the dataset a bit better. You will find some queries below to help you explore the schema. In the process, consider the following questions:

1. Which concepts are modelled in the dataset and how do they relate to each other?
2. The data is stored as tables. Why was this shape chosen and why not the other shapes?
3. In which normal forms are the corresponding relations?
4. What are the efficiency trade-offs from adding an `artist_id` and `artist_name` directly to the `releases` table? Hints:
   - What are some typical queries that would benefit from this change?
   - How often do we need to update artists?
5. What potential problems could result from adding this redundancy?

### Where we got the data from
- [Discogs](https://www.discogs.com/)
- [Discogs XML data dumps](http://data.discogs.com/)
- [Download the dataset](https://cloud.inf.ethz.ch/s/DtjCHTLRHT39BRN/download/discogs.dump.xz) (only necessary if you don't want to use Docker, see `postgres-init.sh` to see how to import it)

### List tables
The following query retrieves a list of tables in the database from a system table describing the current database.

In [4]:
%%sql 
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

UsageError: Cell magic `%%sql` not found.


### List attributes/columns
The following query retrieves a list of columns from the tables in the database.

In [3]:
%%sql 
SELECT table_name, column_name, data_type, is_nullable, ordinal_position
FROM information_schema.columns
WHERE table_schema = 'public'
AND table_name NOT LIKE 'pg_%'
ORDER BY table_name, ordinal_position;

UsageError: Cell magic `%%sql` not found.


## Exercise 2: SQL warm-up
Let us begin with several SQL queries to ease us back into the language.

1. Retrieve all artists with the name of 'Radiohead'.

In [ ]:
%%sql
Select *
FROM artists
Where name = "Radiohead"

2. List the titles of all releases by that artist in alphabetical order.

In [ ]:
%%sql
Select releases.title 
From release
Join released_by using(release_id)
Join artists using(artist_id)
Where artists.name = "Radiohead"
Order By releases.title

3. List the titles of all releases by that artist that contain less than 5 tracks.

In [ ]:
%%sql
Select releases.title
From release
Join released_by using(release_id)
Join artists using(artist_id)
JOIN tracks USING(release_id)
Where artists.name = "Radiohead"
Group By releases.release_id, releases.title
Having Count(*) < 5

4. What are the top 10 artists with the most releases?

In [ ]:
%%sql
Select artists.name, Count(*) as num_releases
From artists
Join released_by using(artist_id)
Join releases using(release_id)
Group By artists.artist_id, artists.name
Order by Count(*) DESC
Limit 10

5. How many artists have more releases than the average number of releases per artist? Please ignore artists with 0 release.

In [ ]:
%%sql

Select artists.name, Count(*) as num_releases
From artists
Join released_by using(artist_id)
Join releases using(release_id)
Group By artists.artist_id, artists.name
Having num_releases > (
    Select AVG(num_releases) from
        (Select artists.name, Count(*) as num_releases
        From artists
        Join released_by using(artist_id)
        Join releases using(release_id)
        Group By artists.artist_id, artists.name
        )
    )

6. What are the names and IDs of the artists that have both a release with the genre 'Pop' *and* a release with the genre 'Classical'? Give a query that uses `INTERSECT` and one that uses `EXISTS`.

In [ ]:
%%sql
Select Distinct(artists.artist_id), artists.name
From artists
Join released_by using(artist_id)
Join releases using(release_id)
where releases.genre = "Pop"
Intersects
Select Distinct(artists.artist_id), artists.name
From artists
Join released_by using(artist_id)
Join releases using(release_id)
where releases.genre = "Classical"

In [ ]:
%%sql
...

## Exercise 3: Impact of release genre on average track duration and track count
For this exercise, we want to find out how average track duration and track count varies across genres.

To start, write a query which finds all of the distinct genres:

In [ ]:
%%sql
Select Distinct genre
From releases

Take a guess as to which genre has:
1. The highest average track count?
2. The lowest average track count?
3. The longest average track duration?
4. The shortest average track duration?

Next, write a query to calculate the average track count per genre:

In [ ]:
%%sql
Select genre, AVG(track_count) as avg_track_count
From (
    Select releases.genre As genre, Count(*) as track_count
    From releases
    Join tracks USING(release_id)
    Group by releases.genre
)
group by genre

In [ ]:
With track_per_release as (
    Select releases.genre, Count(*) as track_count
    From releases
    Join tracks USING(release_id)
    Group by releases.release_id, releases.genre
)

Select genre, AVG(track_count) as avg_track_count
From track_per_release
Group By genre

Write a query to calculate the average duration per genre. Your result should have two attributes: `genre` and `avg_duration`.

In [ ]:
SELECT releases.genre, AVG(tracks.duration) AS avg_duration
FROM releases JOIN tracks USING(release_id)
GROUP BY releases.genre
ORDER BY avg_duration ASC;

Did the results match what you expected? Copy your query into the following python script to plot the result.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Store the result of the query in a Python object (add your query here!)
result = %sql ...

# Convert the result to a Pandas data frame
df = result.DataFrame()

# Extract x and y values for a plot
x = df['genre'].tolist()
y = df['avg_duration'].tolist()

# Print them just for debugging
print(x)
print(y)

# Plot the distribution of scores
fig = plt.figure(figsize =(14, 7))
plt.barh(x, y, align='center')
plt.xlabel('Average Duration (s)')
plt.ylabel('Genre')

## Exercise 4: Discuss query patterns and language features of SQL
1. What patterns did you use in many of the queries above?

2. Do you remember the theory behind them?

3. What makes SQL a declarative language and what advantages does that have?

4. What makes SQL a functional language and what advantages does that have?

## Exercise 5: Limits of SQL (optional)
Explain what the following query does:

In [ ]:
%%sql
WITH RECURSIVE
    X AS (SELECT 3 AS Value),
    artist_releases AS (
        SELECT artists.artist_id, artists.name, releases.release_id, releases.title
        FROM artists, released_by, releases
        WHERE artists.artist_id = released_by.artist_id
        AND released_by.release_id = releases.release_id
    ),
    collaborations AS (
        SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
                ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
        FROM artist_releases AS ar1, artist_releases AS ar2
        WHERE ar1.release_id = ar2.release_id
        AND ar1.artist_id != ar2.artist_id
    ),
    X_hop_collaborations AS (
        SELECT * FROM collaborations  -- base case
        UNION
        SELECT c1.left_id, c1.left_name, c2.right_id, c2.right_name, c1.distance + 1 AS distance
        FROM X_hop_collaborations AS c1
        JOIN collaborations c2 ON c1.right_id = c2.left_id
        WHERE c1.distance < (SELECT * FROM X)
    )
SELECT * 
FROM X_hop_collaborations
WHERE left_name = 'Radiohead'
ORDER BY distance, right_name;